In [4]:
import numpy as np
import mxnet as mx
import sys

image_size = 112
valid_iter = mx.io.ImageRecordIter(
        path_imgrec="/home/vuthede/AI/PFLD-pytorch/data/pfld_test_data.rec", 
        data_shape=(3, image_size, image_size), 
        batch_size=50,
        label_width=205,
        shuffle = False,
        preprocess_threads = 16,
    )

net = gluon.nn.SymbolBlock.imports("/home/vuthede/Downloads/lmks_detector-symbolresnst.json", ['data'], "/home/vuthede/Downloads/lmks_detector-0800resnet.params", ctx=devices)
        
# validate model in test data
NME = 0.0
NUM = 0
for batch in valid_iter:
    data = batch.data[0]
    data = preprocess(data)
    labels = batch.label[0]
    lmks = labels[:, 0:98*2]
    data = data.as_in_context(devices[0])
    lmks = lmks.as_in_context(devices[0])
    shared_feature, regs = net(data)
    regs = nd.Flatten(regs)
    batch_size = data.shape[0]
    NUM += batch_size
    regs = regs.asnumpy()
    lmks = lmks.asnumpy()
    print (regs)
    for i in range(0, batch_size):
        ne = 0.0
        for j in range(60, 76):
            e = (regs[i, j*2 + 0] - lmks[i, j*2 + 0]) * (regs[i, j*2 + 0] - lmks[i, j*2 + 0]) + \
                (regs[i, j*2 + 1] - lmks[i, j*2 + 1]) * (regs[i, j*2 + 1] - lmks[i, j*2 + 1])
            e = np.sqrt(e)
            ne += e
        inter_occular=(lmks[i, 2*60 + 0] - lmks[i, 2*72 + 0]) * (lmks[i, 2*60 + 0] - lmks[i, 2*72 + 0]) +\
                        (lmks[i, 2*60 + 1] - lmks[i, 2*72 + 1]) * (lmks[i, 2*60 + 1] - lmks[i, 2*72 + 1])
        inter_occular = np.sqrt(inter_occular)
        ne = ne / (inter_occular)
        NME += ne

NME /= NUM  
print('Validaton: {} = {}'.format('NME', NME))




NameError: name 'gluon' is not defined

'/home/vuthede/AI/PFLD-pytorch/notebooks'